In [2]:
import pandas as pd 
from sklearn.model_selection import KFold

# data load

In [3]:
train = pd.read_csv('/data/ephemeral/home/level2_klue/KSY/data/train.csv')
back = pd.read_csv('/data/ephemeral/home/level2_klue/KSY/data/back4.csv',index_col = 0)
back = back.iloc[:,:6]

In [4]:
train['증강 여부'] = False
back['증강 여부'] = True

In [6]:
df = pd.concat([train,back])
df = df.reset_index(drop=True)
df.tail()

,id,sentence,subject_entity,object_entity,label,source,증강 여부
33385,32320,"칭기즈 칸, 보르지긴의 일족은 골든 클랜 (Altan Uruk)이라고 불리며, 이들...","{'word': '칭기즈 칸', 'start_idx': 0, 'end_idx': 4...","{'word': '보르지긴', 'start_idx': 11, 'end_idx': 1...",per:other_family,wikipedia,True
33386,32336,헨리 4세 헨리 5세의 아버지는 아들을 일찍 결혼하기 위해 마음을 만들고 있었지만 ...,"{'word': '헨리 4세', 'start_idx': 12, 'end_idx': ...","{'word': '헨리 5세', 'start_idx': 0, 'end_idx': 4...",per:children,wikipedia,True
33387,32404,소냐 옐리치는 1965 년 뉴질랜드의 오클랜드 Dalmatia에서 이민 온 크로아티...,"{'word': '소냐 옐리치', 'start_idx': 0, 'end_idx': ...","{'word': '뉴질랜드', 'start_idx': 14, 'end_idx': 1...",per:place_of_birth,wikipedia,True
33388,32433,"2008 년 11 월 5 일, 드미트리 메드베데프 러시아 대통령은 발트해의 Kali...","{'word': '드미트리 메드베데프', 'start_idx': 14, 'end_i...","{'word': '러시아', 'start_idx': 25, 'end_idx': 27...",per:place_of_residence,wikipedia,True
33389,32463,그의 삼촌은 루이 13세이고 그녀의 남편 인 루이 14세는 사촌 형제가됩니다.,"{'word': '루이 14세', 'start_idx': 21, 'end_idx':...","{'word': '루이 13세', 'start_idx': 8, 'end_idx': ...",per:other_family,wikipedia,True


# data split

In [7]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
train_index, val_index = [],[]
for i, (train_index, val_index) in enumerate(kf.split(X=df,y=df['label'])):
    if i == 0:
        break
len(train_index), len(val_index)

(26712, 6678)

In [8]:
train_df = df.iloc[train_index,:]
val_df = df.iloc[val_index,:]
train_df.shape, val_df.shape

((26712, 7), (6678, 7))

In [9]:
val_df_except_aug = val_df[val_df['증강 여부']==False]
val_df_aug = val_df[val_df['증강 여부']==True]

In [10]:
val_vc = val_df_aug['label'].value_counts()

In [11]:
train_temp = pd.DataFrame()

In [12]:
for label, count in val_vc.items():
    print(label, count) 
    temp = train_df[(train_df['label']==label)&(train_df['증강 여부']==False)].head(count)
    train_temp = pd.concat([train_temp,temp])
    train_df = train_df.drop(temp.index)
train_temp.shape, val_df_aug.shape

per:place_of_birth 28
per:other_family 25
per:place_of_residence 21
per:children 21
per:product 21
per:siblings 17
org:founded_by 15
org:dissolved 12
per:religion 8
per:place_of_death 7
org:political/religious_affiliation 7
per:schools_attended 5
org:number_of_employees/members 2


((189, 7), (189, 7))

In [13]:
train_df = pd.concat([train_df,val_df_aug])
val_df = pd.concat([val_df_except_aug,train_temp])
train_df.shape, val_df.shape

((26712, 7), (6678, 7))

In [14]:
train_df = train_df.iloc[:,:-1]
val_df = val_df.iloc[:,:-1]

In [ ]:
train_df.to_csv("/data/ephemeral/home/level2_klue/KSY/data/train_split2.csv",index=False)
val_df.to_csv("/data/ephemeral/home/level2_klue/KSY/data/validation_split2.csv",index=False)